In [1]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt

#from sklearn import preprocessing as pp
#from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
import re
import time
from tqdm import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

#import torchvision.transforms as transforms
import torch.optim as optimizers
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import WeightedRandomSampler, BatchSampler

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def set_seed(seed: int = 123):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
set_seed(123)

In [4]:
all_new = pd.read_csv('data/feature_df_agg_svd16.csv')
BERT_svd256 = pd.read_csv("data/BERT_svd256.csv")
ALBERT_svd256 = pd.read_csv("data/ALBERT_svd256.csv")
ROBERTA_svd256 = pd.read_csv("data/ROBERTA_svd256.csv")
XLMROBERTA_svd256 = pd.read_csv("data/XLMROBERTA_svd256.csv")
tfidf_svd_raw_128 = pd.read_csv("data/tfidf_svd_raw_128_2gram.csv")
mfw_df = pd.read_csv('data/mfw.csv')
miw_df = pd.read_csv('data/miw.csv')

In [5]:
all_new = pd.merge(all_new, BERT_svd256, on="id", how="outer")
all_new = pd.merge(all_new, ALBERT_svd256, on="id", how="outer")
all_new = pd.merge(all_new, ROBERTA_svd256, on="id", how="outer")
all_new = pd.merge(all_new, XLMROBERTA_svd256, on="id", how="outer")
all_new = pd.merge(all_new, tfidf_svd_raw_128, on="id", how="outer")
all_new = pd.merge(all_new, miw_df, on="id", how="outer")
all_new = pd.merge(all_new, mfw_df, on="id", how="outer")

In [6]:
test_data = all_new[all_new["data_type"] == "test"]
train_data = all_new[all_new["data_type"] == "train"]

In [7]:
class CFDataset(Dataset):
    def __init__(self, csv_file, feature_cols, transform=None):
        self.csv_file = csv_file
        self.transform = transform
        self.feature_cols = feature_cols
        self.target_cols = ["state"]
        
        
    def __len__(self):
        return len(self.csv_file)
    
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        features = self.csv_file[self.feature_cols].iloc[idx]
        features = torch.FloatTensor(features)
        
        target = self.csv_file[self.target_cols].iloc[idx]
        target = torch.tensor(target)
        
        if self.transform:
            features = self.transform(features)
            
        return features, target

In [8]:
class CFDataModule(pl.LightningDataModule):
    def __init__(self, train, validation, feature_cols, transform, batch_size, num_workers):
        super().__init__()
        self.csv_file = train
        self.transform = transform
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.validation = validation
        self.feature_cols = feature_cols
        

    def setup(self, stage=None):
        self.train_dataset = CFDataset(csv_file=self.csv_file, feature_cols=self.feature_cols, transform=self.transform)
        self.val_dataset = CFDataset(csv_file=self.validation, feature_cols=self.feature_cols, transform=self.transform)
            
        
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          #pin_memory=True,
                         )
    
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          drop_last=True,
                          num_workers=self.num_workers,
                          #pin_memory=True,
                         )

In [9]:
class CFModule(pl.LightningModule):
    def __init__(self, input_dim, hidden_sizes, out_dim, dropout):
        super(CFModule, self).__init__()
        self.layers = []
        self.dropouts = []
        self.dense_layers = []
        
        for hidden_size in hidden_sizes:
            self.layers.append(nn.BatchNorm1d(input_dim))
            self.dropouts.append(nn.Dropout(dropout))
            self.dense_layers.append(nn.utils.weight_norm(nn.Linear(input_dim, hidden_size)))
            input_dim = hidden_size
        self.layers.append(nn.Linear(input_dim, out_dim))
        
        for idx, layer in enumerate(self.layers):
            setattr(self, "fc{}".format(idx), layer)

        for idx, dropout in enumerate(self.dropouts):
            setattr(self, "drop{}".format(idx), dropout)
            
        for idx, dense in enumerate(self.dense_layers):
            setattr(self, "dense{}".format(idx), dense)

    
    def forward(self, x):
        for batch_norm, dropout, dense in zip(self.layers, self.dropouts, self.dense_layers):
            x = batch_norm(x)
            x = dropout(x)
            x = F.relu(dense(x))    
        x = self.layers[-1](x)
        
        return torch.sigmoid(x)
    
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        # you should define log as {"tag_name/log_name"}
        tensorboard_logs = {'train/train_loss': loss, "train/train_acc": acc}
        return {"loss": loss, "acc": acc, "logs": tensorboard_logs, "progress_bar": tensorboard_logs}
    
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        loss = self.criterion(pred, t)
        acc = self.metric(pred, t)
        logs = {"val_loss": loss, "val_acc": acc}
        self.log("val_loss", loss)
        return {"val_loss": loss, "val_acc": acc, "progress_bar": logs}

    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([torch.tensor(x['val_acc']) for x in outputs]).mean()
        tensorboard_logs = {'val/avg_loss': avg_loss, "val/avg_acc": avg_acc}
        print(f"val_loss: {avg_loss}, val_acc: {avg_acc}")
        # you should call back as name "val_loss" to using the Early-Stopping
        return {'val_loss': avg_loss, "val_acc": avg_acc, 'log': tensorboard_logs}
    
    
    def test_step(self, batch, batch_idx):
        x, t = batch
        pred = self.forward(x)
        return {"preds": pred}
    
    
    def test_epoch_end(self, outputs):
        preds = torch.cat([x["preds"] for x in outputs], dim=0).squeeze()
        preds = np.array(preds.tolist())
        return {"preds": preds}
    
    
    def configure_optimizers(self):
        optimizer = optimizers.Adam(self.parameters(), 
                                    lr=0.01, betas=(0.9,0.999),
                                    eps=1e-08, weight_decay=0, amsgrad=False)
        scheduler = {"scheduler": 
                     optimizers.lr_scheduler.CosineAnnealingLR(
                        optimizer, T_max=10),
                    "interval": "epoch",
                    "monitor": "val_loss"}
        return [optimizer], [scheduler]
    
    
    def criterion(self, pred, t):
        pred = pred.float()
        t = t.float()
        #return F.binary_cross_entropy(input=pred, target=t)
        return F.mse_loss(input=pred, target=t)
    
    
    def metric(self, pred, t):
        t = t.to('cpu')
        pred = np.array(pred.tolist())
        pred = np.where(pred<0.5, 0, 1)
        return f1_score(y_true=t, y_pred=pred, average='binary', sample_weight=None, zero_division='warn')

In [10]:
def add_cols_kfold(n_folds, train, y_train, seed):
    cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    
    for fold, (t_idx, v_idx) in enumerate(cv.split(train, y_train)):
        train.loc[v_idx, 'kfold'] = int(fold)
        
    return train

In [11]:
def run_nn(optuna_config_dict, output_path):
    # trainer config
    max_epochs = 100
    
    
    # data module config
    #train_set
    transform = None
    batch_size = 256*8
    num_workers = 4
    
    # model config
    feature_cols = optuna_config_dict["selected_feature"]
    params = optuna_config_dict["best_params"]
    n_layers = params["n_layers"]
    dropout = params["dropout"]
    hidden_sizes = [0]*n_layers
    for n in range(n_layers):
        hidden_sizes[n] = params[f"n_units_l{n}"]
    input_dim = len(feature_cols)
    out_dim = 1
    
    # early stopping config
    patience = 4
    
    # kfold config
    n_folds = 5
    seed = optuna_config_dict["seed"]
    
    config_dict = {
        "max_epochs": max_epochs,
        "output_path": output_path,
        "batch_size": batch_size,
        "num_workers": num_workers,
        "optuna_config" : optuna_config_dict,
        "patience": patience,
        "n_folds": n_folds,
        "seed": seed 
    }
    
    # prepare data for with kfold
    train = add_cols_kfold(n_folds=n_folds,
                            train=train_data,
                           y_train=train_data["category2"],
                            seed=seed)
    
    test_prediction = []
    oof_train = np.zeros((len(train_data),))
    for fold in train.kfold.unique():
        train_fold = train[train["kfold"] != fold]
        val_fold = train[train["kfold"] == fold]
        val_idx = val_fold.index
    
        cf = CFDataModule(train=train_fold, validation=val_fold, feature_cols=feature_cols,
                          transform=transform, batch_size=batch_size, num_workers=num_workers)
        model = CFModule(input_dim=input_dim, hidden_sizes=hidden_sizes, out_dim=out_dim, dropout=dropout)

        early_stopping = EarlyStopping('val_loss', patience=patience, verbose=True)
        trainer = Trainer(
            max_epochs=max_epochs,
            weights_save_path=output_path,
            gpus = 1 if torch.cuda.is_available() else None,
            callbacks=[early_stopping],
            #accumulate_grad_batches=1
            # use_amp=False,
            )
        
        # training
        trainer.fit(model, cf)
        
        # oof prediction
        oof_dataset = CFDataset(val_fold, feature_cols)
        oof_dataloader = DataLoader(oof_dataset,
                                    batch_size=batch_size,
                                    drop_last=False,
                                    shuffle=False,
                                    num_workers=num_workers,
                                    pin_memory=True)
        oof_results = trainer.test(test_dataloaders=oof_dataloader, verbose=False)
        oof_train[val_idx] = oof_results[0]["preds"]
        
        # test prediction
        test_dataset = CFDataset(test_data, feature_cols)
        test_dataloader = DataLoader(test_dataset,
                                    batch_size=batch_size,
                                    drop_last=False,
                                     shuffle=False,
                                    num_workers=num_workers,
                                    pin_memory=True)
        test_results = trainer.test(test_dataloaders=test_dataloader, verbose=False)
        test_preds = test_results[0]["preds"]
        test_prediction.append(test_preds)
        torch.cuda.empty_cache()
        
    return oof_train, sum(test_prediction)/len(test_prediction), config_dict

In [12]:
if __name__ == "__main__":
    
    feature_names = [
        "ALBERT",
        "BERT",
        "ROBERTA",
        "XLMROBERTA",
        "tfidf",
        "tfidf+ALBERT",
        "tfidf+BERT",
        "tfidf+ROBERTA",
        "tfidf+XLMROBERTA",
    ]
    
    for feature_name in feature_names:
        with open(f"data/optuna_nn_mse_{feature_name}_seed2024.json") as f:
            optuna_config_dict = json.load(f)
        
        output_path = f"./nn_logs/{feature_name}_mse"
        oof, preds, config_dict = run_nn(optuna_config_dict, output_path)
        
        config_dict["out_of_fold"] = list(oof)
        config_dict["prediction"] = list(preds)
        with open(f"data/nn_mse_{feature_name}_seed2024.json", 'w') as f:
            json.dump(config_dict, f)

EarlyStopping mode set to min for monitoring val_loss.
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name   | Type        | Params
----------------------------------------
0  | fc0    | BatchNorm1d | 992   
1  | fc1    | BatchNorm1d | 48    
2  | fc2    | BatchNorm1d | 32    
3  | fc3    | BatchNorm1d | 16    
4  | fc4    | Linear      | 116   
5  | drop0  | Dropout     | 0     
6  | drop1  | Dropout     | 0     
7  | drop2  | Dropout     | 0     
8  | drop3  | Dropout     | 0     
9  | dense0 | Linear      | 12.0 K
10 | dense1 | Linear      | 416   
11 | dense2 | Linear      | 144   
12 | dense3 | Linear      | 1.2 K 
----------------------------------------
14.9 K    Trainable params
0         Non-trainable params
14.9 K    Total params


val_loss: 0.59228515625, val_acc: 0.06476484194294525


val_loss: 0.23202982544898987, val_acc: 0.013645224171539962


val_loss: 0.1951092779636383, val_acc: 0.6959280803807509


val_loss: 0.17621923983097076, val_acc: 0.7457627118644069


...

val_loss: 0.13929228484630585, val_acc: 0.8067070330693992


val_loss: 0.13949993252754211, val_acc: 0.8092807424593967


val_loss: 0.14184829592704773, val_acc: 0.8052895576835385


val_loss: 0.1421903371810913, val_acc: 0.8130155820348304


val_loss: 0.1412237584590912, val_acc: 0.8106402164111811

